In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [ ]:
# !pip install pydicom
# !pip install PyWavelets

In [ ]:
import os
import pandas as pd
import pydicom
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import pywt

In [ ]:
dataset_path = "/content/drive/MyDrive/6810_Project/datasets/archive/overview.csv"

dicom_dir = "/content/drive/MyDrive/6810_Project/datasets/archive/dicom_dir"

In [ ]:
# Load the metadata CSV file
metadata = pd.read_csv(os.path.join(dataset_path))
# print("Dataset Overview:")
# print(metadata.head())

In [ ]:
# Function to load a DICOM image
def load_dicom_image(file_path):
    dicom = pydicom.dcmread(file_path)
    image = dicom.pixel_array
    return image

In [ ]:
# Function to preprocess image (resize and normalize)
def preprocess_image(image, target_size=(256, 256)):
    image_resized = cv2.resize(image, target_size, interpolation=cv2.INTER_AREA)
    image_normalized = image_resized / np.max(image_resized, initial=1)  # Avoid division by zero
    return image_normalized

In [ ]:
# Example: Load and preprocess a sample image
sample_row = metadata.iloc[0]
dicom_filename = sample_row['dicom_name']  # Adjust column name if different
dicom_path = os.path.join(dicom_dir, dicom_filename)

# Load and verify the DICOM image
raw_image = load_dicom_image(dicom_path)
print(f"Raw Image Shape: {raw_image.shape}")
print("Raw Image Sample Data:", raw_image[:5, :5])  # Check some pixel values

# Preprocess the image
processed_image = preprocess_image(raw_image)
print(f"Processed Image Shape: {processed_image.shape}")
print("Processed Image Sample Data:", processed_image[:5, :5])


In [ ]:
# Visualize the original and processed images
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.title("Original Image")
plt.imshow(raw_image, cmap='gray')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.title("Processed Image (Resized & Normalized)")
plt.imshow(processed_image, cmap='gray')
plt.axis('off')

plt.show(block=True)  # Ensure the plot window stays open
print("Plot should be displayed now.")

In [ ]:
# Standard 2D-DWT function
def dwt_2d(image):
    coeffs = pywt.wavedec2(image, 'haar', level=1)
    coeff_arr, coeff_slices = pywt.coeffs_to_array(coeffs)
    return coeff_arr, coeff_slices

# Inverse Standard 2D-DWT function
def idwt_2d(coeff_arr, coeff_slices):
    coeffs = pywt.array_to_coeffs(coeff_arr, coeff_slices, output_format='wavedec2')
    return pywt.waverec2(coeffs, 'haar')

# Function to perform DWT and return sub-bands
def dwt_2d_with_subbands(image, wavelet='haar', level=1):
    coeffs = pywt.wavedec2(image, wavelet, level=level)
    # For level=1, coeffs[0] is approximation, coeffs[1] is a tuple of (horizontal, vertical, diagonal) details
    cA = coeffs[0]  # Approximation (low-frequency)
    (cH, cV, cD) = coeffs[1]  # Detail coefficients: Horizontal, Vertical, Diagonal
    return cA, cH, cV, cD

In [ ]:
coeff_arr, coeff_slices = dwt_2d(processed_image)
print(f"DWT Coefficient Array Shape: {coeff_arr.shape}")

# Reconstruct the image from DWT coefficients
reconstructed_image = idwt_2d(coeff_arr, coeff_slices)
print(f"Reconstructed Image Shape: {reconstructed_image.shape}")

In [ ]:
# Visualize the original, processed, and DWT-reconstructed images
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.title("Original Image")
plt.imshow(raw_image, cmap='gray')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.title("Processed Image")
plt.imshow(processed_image, cmap='gray')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.title("DWT Reconstructed Image")
plt.imshow(reconstructed_image, cmap='gray')
plt.axis('off')

plt.show()
print("Plots should be displayed now.")

In [ ]:
'''
Instead of returning a single coefficient array, this function extracts the individual features:
            cA: Reconstructs the overall structure of the image. (smoothed version of the image, low-frequency).
            cH: Emphasizes horizontal edges (e.g., lines or boundaries running left-right).
            cV: Emphasizes vertical edges (e.g., lines or boundaries running up-down).
            cD: Emphasizes diagonal edges or textures.
        These sub-bands show what the DWT is "detecting" in terms of frequency components.
    Shape Information: Each sub-band is typically half the size of the input image

    If DWT is Working:
        The approximation (cA) should look like a downscaled, smoother version of the original image.
        The detail coefficients (cH, cV, cD) should highlight edges or changes in intensity:
            Bright areas in cH indicate strong horizontal edges.
            Bright areas in cV indicate strong vertical edges.
            Bright areas in cD indicate strong diagonal edges.
        If these sub-bands are mostly flat or zero, it might mean the image lacks significant edges or the normalization flattened the data too much.
    Highlighting Regions:
        The sub-bands themselves act as "highlight maps" for the regions detected by DWT. For example, if cH shows bright spots, those correspond to horizontal features in the original image.
'''

In [ ]:
# Number of images to visualize
num_images_to_display = 3

# Process all images in the dataset
for idx, row in metadata.iterrows():
    dicom_filename = row['dicom_name']  # Adjust column name if different
    dicom_path = os.path.join(dicom_dir, dicom_filename)

    try:
        # Load and preprocess the DICOM image
        raw_image = load_dicom_image(dicom_path)
        processed_image = preprocess_image(raw_image)

        # Apply DWT and get sub-bands
        cA, cH, cV, cD = dwt_2d_with_subbands(processed_image)

        # Visualize only the first num_images_to_display images
        if idx < num_images_to_display:
            print(f"Processing Image {idx + 1}: {dicom_filename}")
            print(f"Raw Image Shape: {raw_image.shape}")
            print(f"Processed Image Shape: {processed_image.shape}")
            print(f"Approximation (cA) Shape: {cA.shape}")
            print(f"Horizontal Detail (cH) Shape: {cH.shape}")
            print(f"Vertical Detail (cV) Shape: {cV.shape}")
            print(f"Diagonal Detail (cD) Shape: {cD.shape}")

            # Create a figure to display original image and DWT sub-bands
            plt.figure(figsize=(20, 5))

            # Original Image
            plt.subplot(1, 5, 1)
            plt.title(f"Original Image {idx + 1}")
            plt.imshow(processed_image, cmap='gray')
            plt.axis('off')

            # Approximation (Low-frequency component)
            plt.subplot(1, 5, 2)
            plt.title("Approximation (cA)")
            plt.imshow(cA, cmap='gray')
            plt.axis('off')

            # Horizontal Detail (High-frequency horizontal edges)
            plt.subplot(1, 5, 3)
            plt.title("Horizontal Detail (cH)")
            plt.imshow(cH, cmap='gray')
            plt.axis('off')

            # Vertical Detail (High-frequency vertical edges)
            plt.subplot(1, 5, 4)
            plt.title("Vertical Detail (cV)")
            plt.imshow(cV, cmap='gray')
            plt.axis('off')

            # Diagonal Detail (High-frequency diagonal edges)
            plt.subplot(1, 5, 5)
            plt.title("Diagonal Detail (cD)")
            plt.imshow(cD, cmap='gray')
            plt.axis('off')

            plt.tight_layout()
            plt.show()
            print(f"DWT sub-bands for Image {idx + 1} displayed.")

        else:
            # For images beyond the display limit, just process without plotting
            print(f"Processed Image {idx + 1}: {dicom_filename} (No plot)")

    except Exception as e:
        print(f"Error processing {dicom_filename}: {str(e)}")

print("All images processed.")